<a href="https://colab.research.google.com/github/JohnMichaelCourville/Forest-Cover-Type-Classification/blob/main/CoverType_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  InputLayer
from tensorflow.keras.layers import  Dense
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
data = pd.read_csv('cover_data.csv')
data.head()
data['class'].value_counts()


2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: class, dtype: int64

In [ ]:

#remove classification column from feature set
X = data.loc[:,:'Soil_Type40']

print(X.head())

#create labels set of claissication column
y = data.loc[:,'class']

print(y.head())
## Wanted to use K fold stratification and then averaged out my scores but,
## I had trouble excecuting. I gathers that the simple stratification in the test 
## train split was sufficient since both the random forest and neural net performed so well. 

# skf = StratifiedKFold(n_splits=5)
# skf.get_n_splits(X, y)
# print(skf)
# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

# split and statify(becasue of unbalanced classes)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, stratify = y)




   Elevation  Aspect  Slope  ...  Soil_Type38  Soil_Type39  Soil_Type40
0       2596      51      3  ...            0            0            0
1       2590      56      2  ...            0            0            0
2       2804     139      9  ...            0            0            0
3       2785     155     18  ...            0            0            0
4       2595      45      2  ...            0            0            0

[5 rows x 54 columns]
0    5
1    5
2    2
3    2
4    5
Name: class, dtype: int64


In [ ]:

#convert labels to one-hot endoded labels in order to use categoricalcrossentropy

y_train = tf.keras.utils.to_categorical(y_train, dtype = 'int64')
y_test = tf.keras.utils.to_categorical(y_test, dtype = 'int64')
x_train.head()

#Z score standardization
ct = StandardScaler()

x_train = ct.fit_transform(x_train)
x_test = ct.transform(x_test)
#print(x_train)

model_forest = RandomForestClassifier()
model_forest.fit(x_train, y_train)
model_forest.score(x_test, y_test)


0.9457384798971911

In [ ]:
y_predict = model_forest.predict(x_test)

print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.97      0.94      0.95     63552
           2       0.95      0.97      0.96     84991
           3       0.95      0.95      0.95     10726
           4       0.94      0.81      0.87       824
           5       0.95      0.70      0.81      2848
           6       0.95      0.85      0.90      5210
           7       0.97      0.94      0.96      6153

   micro avg       0.96      0.95      0.95    174304
   macro avg       0.84      0.77      0.80    174304
weighted avg       0.96      0.95      0.95    174304
 samples avg       0.95      0.95      0.95    174304



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model = Sequential()

model.add(InputLayer(x_train.shape[1],))

model.add(Dense(512, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(8, activation = 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               28160     
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 16)                1040      
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dense_5 (Dense)             (None, 8)                 72        
                                                                 
Total params: 103,328
Trainable params: 103,328
Non-trai

In [ ]:
EPOCHS = 15
BATCH_SIZE = 128

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ["accuracy"])

model.fit(x_train, y_train, epochs = EPOCHS, batch_size = BATCH_SIZE)

Epoch 1/15
3178/3178 [==============================] - 22s 7ms/step - loss: 0.5491 - accuracy: 0.7680
Epoch 2/15
3178/3178 [==============================] - 20s 6ms/step - loss: 0.3992 - accuracy: 0.8334
Epoch 3/15
3178/3178 [==============================] - 20s 6ms/step - loss: 0.3367 - accuracy: 0.8614
Epoch 4/15
3178/3178 [==============================] - 20s 6ms/step - loss: 0.2986 - accuracy: 0.8779
Epoch 5/15
3178/3178 [==============================] - 21s 7ms/step - loss: 0.2730 - accuracy: 0.8894
Epoch 6/15
3178/3178 [==============================] - 22s 7ms/step - loss: 0.2546 - accuracy: 0.8966
Epoch 7/15
3178/3178 [==============================] - 21s 7ms/step - loss: 0.2404 - accuracy: 0.9028
Epoch 8/15
3178/3178 [==============================] - 22s 7ms/step - loss: 0.2291 - accuracy: 0.9075
Epoch 9/15
3178/3178 [==============================] - 21s 7ms/step - loss: 0.2192 - accuracy: 0.9113
Epoch 10/15
3178/3178 [==============================] - 21s 7ms/step - l

In [ ]:
y_estimate = model.predict(x_test)

y_estimate = np.argmax(y_estimate, axis = 1)

y_true = np.argmax(y_test, axis = 1)

print(classification_report(y_true, y_estimate))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92     63552
           2       0.93      0.95      0.94     84991
           3       0.92      0.91      0.92     10726
           4       0.84      0.79      0.81       824
           5       0.83      0.72      0.77      2848
           6       0.83      0.88      0.85      5210
           7       0.94      0.92      0.93      6153

    accuracy                           0.93    174304
   macro avg       0.89      0.87      0.88    174304
weighted avg       0.93      0.93      0.93    174304

